In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

cwd = os.getcwd() #
fullscrn()

In [2]:
def wndwng0d(x):            #روش تشخیص ایپاک پایه
    smpl_rte=480
    i=0
    wndws=np.array([])
    #for i in range (1,2):   #print(np.shape(x))
    xj=np.array(x)
    peaks,_ = find_peaks(xj, height=0.02*max(xj), distance=160,prominence=200, width=3)
    if len(peaks)<6 :
        peaks,_ = find_peaks(xj, height=0.01*max(xj), distance=120,prominence=60, width=3)
    if len(peaks)>1 :
        for z in range (len(peaks)):
            if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                win=np.array(xj[peaks[z]:peaks[z+1]])
                win = signal.resample(win, smpl_rte)
                if len(win)==smpl_rte :
                    wndws=np.append(wndws, win)
    rows=int(len(wndws)/(smpl_rte))            #/(smpl_rte+1))
    wndws0=np.reshape(wndws,(rows,(smpl_rte)))    #(wndws,(rows,(smpl_rte+1)))
    return(wndws0)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=17
for i in range (1,18):
    vars()['ecg'+str(i)]=MITBIH(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,18):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=1000

1 283	2 66	3 20	4 135	5 13	6 21	7 133	8 55	9 13	10 10	11 10	12 10	13 11	14 103	15 62	16 10	17 45	

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])
for cls in range (1,18):                                 #ساخت پنجره های داده های آموزش اصلی
    dta=np.array(vars()['ecg'+str(cls)][int(.1*len(vars()['ecg'+str(cls)])):,:]) #انتخاب محدوده ترِین جهت داده افزایی
    vars()['wndws'+str(cls)]=wndwng(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    clm0vlu=np.zeros(len(vars()['wndws'+str(cls)]))
    cls_clm=np.array(clm0vlu)
    cls_clm=np.int16(cls_clm+cls)
    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    vars()['wndws'+str(cls)]=np.concatenate((vars()['wndws'+str(cls)],cls_clm),axis=1)   

print("each class and its windows =")
cls_wndws=np.array([])
for cls in range (1,18):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
cls_wndws=np.int16(cls_wndws)    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 2598	2 788	3 224	4 1564	5 158	6 189	7 1371	8 553	9 114	10 104	11 72	12 107	13 156	14 1014	15 633	16 45	17 450	
 max = 2598


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [5]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,7), int) 
rslts=np.array([])
for X in range(1,5):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          
    f_scr=np.array([57,57,57,57,74,57,57,57])     #alpha_trim3_UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
    #f_scr=np.array([57,65,74,75,74,62,60,74])    #alpha_Base_UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
    #f_scr=np.array([51,68,73,74,71,56,55,70])    #Old_Old_UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
    f_scr_difrnc=f_scr-f_scr[0]
    f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
    aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 
    for i in range (1,cls_num+1):
        aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws-cls_wndws[i-1])],axis=0)
        #vlum_win=mx_wndws-cls_wndws[i-1]                        #میزان داده افزایی در روش های عمیق (پنجره)
    aug_amnt=np.int16(np.round(aug_amnt))
    aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
    #print(np.int16(100*f_scr_rtio))
    #print(aug_amnt)
    rprt=np.empty((0,3),float)
    print(aug_amnt)
    for repeat in range(1,rpt_cnn_clsfctn+1):
        xtrain=np.empty((0,smpl_rte+1), float) 
        for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
            xtrain=np.append(xtrain,Aug_data(cwd,i,'00',cls_wndws[i-1]),axis=0)
            if aug_amnt[i-1,0]>0:
                methd=10
                sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                xtrain=np.append(xtrain,Aug_data(cwd,i,10,aug_amnt[i-1,0]),axis=0)
            if aug_amnt[i-1,1]>0:
                methd=20
                sort=-3
                dp_slct_dstrb=2
                xtrain=np.append(xtrain,Aug_data(cwd,i,20,aug_amnt[i-1,1]),axis=0)
            if aug_amnt[i-1,2]>0:
                methd=30
                sort=-3
                dp_slct_dstrb=4
                xtrain=np.append(xtrain,Aug_data(cwd,i,30,aug_amnt[i-1,2]),axis=0)
            if aug_amnt[i-1,3]>0:
                methd=40
                sort=-1
                dp_slct_dstrb=4
                xtrain=np.append(xtrain,Aug_data(cwd,i,40,aug_amnt[i-1,3]),axis=0)
            if aug_amnt[i-1,4]>0:
                methd=50
                sort=-3
                dp_slct_dstrb=4
                xtrain=np.append(xtrain,Aug_data(cwd,i,50,aug_amnt[i-1,4]),axis=0)
            if aug_amnt[i-1,5]>0:
                methd=60
                sort=-3
                dp_slct_dstrb=4                             
                xtrain=np.append(xtrain,Aug_data(cwd,i,60,aug_amnt[i-1,5]),axis=0) #Scale
            if aug_amnt[i-1,6]>0:
                methd=70
                sort=-3
                dp_slct_dstrb=4
                xtrain=np.append(xtrain,Aug_data(cwd,i,70,aug_amnt[i-1,6]),axis=0)

        mx_aug=np.max(xtrain[:,:-1])
        mx=np.max(ecg[:,:-1])
        mn_aug=np.min(xtrain[:,:-1])
        mn=np.min(ecg[:,:-1])

        #for i in range (1,18):             #نرمالسازی داده های افزایشی
        xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

        wndws_test=np.empty((0,smpl_rte+1), float)
        smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
        i=0
        windws=np.array([])
        for cls in range (1,18):                                 #ساخت پنجره های داده های آموزش اصلی
            dta=np.array(vars()['ecg'+str(cls)][:int(.1*len(vars()['ecg'+str(cls)])),:]) #انتخاب محدوده ترِین جهت داده افزایی
            vars()['wndws_tst'+str(cls)]=wndwng(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
            clm0vlu=np.zeros(len(vars()['wndws_tst'+str(cls)]))
            cls_clm=np.array(clm0vlu)
            cls_clm=np.int16(cls_clm+cls)
            cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
            vars()['wndws_tst'+str(cls)]=np.concatenate((vars()['wndws_tst'+str(cls)],cls_clm),axis=1)   

        #print(np.max(wndws_tst))
        for i in range (1,18):                # Normalization test windows
            cls=i
            '''print('\n cls', i, ' >> ')
            print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
            print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))'''
            vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
            '''print('after normalizing >>')
            print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
            print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))'''

        #print("each class and its windows =")                    #تجمیع کلاس های تست
        cls_wndws=np.array([])
        for cls in range (1,18):                                # آرایه ی تعداد پنجره ی هر کلاس
            wns=len(vars()['wndws_tst'+str(cls)])
            cls_wndws=np.append(cls_wndws,wns)
            #print(cls, wns, end='\t')
            wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)

        #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
        #print('\n max instance in classes of test windows =', mx_wndws)

        Xtest=wndws_test[:,:-1]
        ytest=np.int16(wndws_test[:,-1])

        trainx=np.random.permutation(xtrain)
        Xtrain=np.array(trainx[:,:-1])
        ytrain=np.int16(trainx[:,-1])

        X_train=np.array(Xtrain)
        y_train=np.array(ytrain)
        X_test=np.array(Xtest)
        y_test=np.array(ytest)
        X_valid=np.array(Xtrain)
        y_valid=np.array(ytrain)

        y_train_cat=to_categorical(y_train)
        y_valid_cat=to_categorical(y_valid)
        y_test_cat=to_categorical(y_test)
        X_train = np.expand_dims(X_train, axis=2)
        X_valid = np.expand_dims(X_valid, axis=2)
        X_test = np.expand_dims(X_test, axis=2)

        #VGG16_1D

        accuracy=0
        acc_crs=np.array([])
        ##for i in range (10):
        ##    if accuracy<0.2 :

        verbose, epochs, batch_size = 0, 1000, btch
        n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
        steps_per_epoch = len(X_train)//batch_size
        validation_steps = len(X_valid)//batch_size # if you have test data



        model_crs = Sequential()
        #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
        model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
        model_crs.add(BatchNormalization())
        model_crs.add(MaxPooling1D(pool_size=2,strides=3))

        model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
        model_crs.add(BatchNormalization())
        model_crs.add(MaxPooling1D(pool_size=2,strides=2))

        model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
        model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
        model_crs.add(MaxPooling1D(pool_size=2,strides=2))

        model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
        model_crs.add(MaxPooling1D(pool_size=2,strides=2))

        model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
        #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


        model_crs.add(Flatten())
        model_crs.add(Dropout(0.1))
        model_crs.add(Dense(units=512,activation="relu"))
        #model_crs.add(Dense(units=2048,activation="relu"))
        model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

        model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
        # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

        earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

        model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

        #_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

        #print('np.shape(X_test)=',np.shape(X_test))

        #print('Accuracy= ', accuracy)

        #y_pred_vgg = model_crs.predict_classes(X_test)
        predict_x=model_crs.predict(X_test)              # Function 1

        y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

        #y_pred_crs = model_crs.predict_classes(X_test)

        cm = confusion_matrix(y_test, y_pred_crs)
        #print(cm)
        acc3=accuracy_score(y_test, y_pred_crs)

        #print(classification_report(y_test, y_pred_crs))

        #sns.heatmap(cm, annot=True)

        #plt.imshow(cm)

        ### Maxcount Predicting Test data
        ecg_test=np.array(ecg1[:int(.1*len(ecg1)),:])
        #print(1,len(ecg_test), end='\t')
        for i in range (2,18):
            ecg_test=np.concatenate((ecg_test,vars()['ecg'+str(i)][:int(.1*len(vars()['ecg'+str(i)])),:]),axis=0)
            #print(i,len(vars()['ecg'+str(i)]), end='\t')    
        #print('\n ecg_test shape = ',np.shape(ecg_test))
        i=0
        mlpinputshps=np.array([])
        maxcntlbl=np.array([])
        mlpinput=np.array([])
        mlplbl=np.array([])
        cnnlbl=np.array([])
        peaklens=np.array([])            #array of peaks quantity in each raw signal
        x=np.array([])
        wow=0

        for i in range (len(ecg_test)): #calculating max peaks in row signals
            peaklens=np.append(peaklens,len(wndwng0d(ecg_test[i])))
        maxpeaks=int(np.max(peaklens))
        #print(maxpeaks)

        prd_lbls_tst=np.empty((0,maxpeaks), int)              #have peaks(windows) lables for each raw signal

        for i in range (len(ecg_test)):
            windws=np.array([])
            yyy=np.array([])
            yy=np.array([])
            win=np.array([])
            x=np.array(ecg_test[i,:-1])
            windws=wndwng0d(x)
            windws= 2*(windws-mn_aug)/(mx_aug - mn_aug) - 1
            windws = np.expand_dims(windws, axis=2)
            yy= model_crs.predict(windws)#yy= Predict each window lable for a raw signal (3600 Pnt) (17 prob in row)
            yyy=np.int16(maxindx(yy))         # one-hot to class lable
            maxcntlbl=np.append(maxcntlbl,(np.bincount(yyy).argmax()))
            zers=np.zeros(int(maxpeaks-len(yyy)))
            x=np.array(np.append(yyy, zers))
            prd_lbls_tst=np.append(prd_lbls_tst, [x], axis=0)
        maxcntlbl=np.int16(maxcntlbl)                           
        maxcntlbl=np.int16(maxcntlbl)

        lbls=np.int16(ecg_test[:,-1])

        '''cm = confusion_matrix(lbls, maxcntlbl)
        print(cm)
        acc3=accuracy_score(lbls, maxcntlbl)
        print(classification_report(lbls, maxcntlbl))

        predict_x=model_crs.predict(X_test)# Function 1
        y_pred_crs=maxindx(predict_x)      # function from augment.py to remove 0 index predictions'''

        rprt0=classification_report(lbls, maxcntlbl,output_dict=True)
        rprt_row=np.array([])
        rprt_row=np.append(rprt_row,rprt0['accuracy'])
        rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
        rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
        rprt=np.append(rprt,[rprt_row],axis=0)
    #print('rprt=',rprt)
    maxm=rprt[np.argmax(rprt[:,1])]
    #print('maxm=',maxm)
    avrg=np.mean(rprt,axis=0)
    #print('avrg=',avrg)
    rslts=np.append(rslts,['X '+str(X)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
    print('rslts=',rslts[-1])


 max = 2599
[[   0    0    0    0    0    0    0]
 [   0    0    0 1810    0    0    0]
 [   0    0    0 2374    0    0    0]
 [   0    0    0 1034    0    0    0]
 [   0    0    0 2440    0    0    0]
 [   0    0    0 2409    0    0    0]
 [   0    0    0 1227    0    0    0]
 [   0    0    0 2045    0    0    0]
 [   0    0    0 2484    0    0    0]
 [   0    0    0 2494    0    0    0]
 [   0    0    0 2526    0    0    0]
 [   0    0    0 2491    0    0    0]
 [   0    0    0 2442    0    0    0]
 [   0    0    0 1584    0    0    0]
 [   0    0    0 1965    0    0    0]
 [   0    0    0 2553    0    0    0]
 [   0    0    0 2148    0    0    0]]
1/1 [==============================] - 0s 13ms/step


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 13ms/step


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 13ms/step
rslts= X 1  average= [71 70 70]  max= [72 71 70]  var= [3 0 1]
[[  0   0   0 310   0   0   0]
 [  0   0   0 553   0   0   0]
 [  0   0   0 586   0   0   0]
 [  0   0   0 467   0   0   0]
 [  0   0   0 602   0   0   0]
 [  0   0   0 600   0   0   0]
 [  0   0   0 460   0   0   0]
 [  0   0   0 563   0   0   0]
 [  0   0   0 611   0   0   0]
 [  0   0   0 611   0   0   0]
 [  0   0   0 615   0   0   0]
 [  0   0   0 604   0   0   0]
 [  0   0   0 612   0   0   0]
 [  0   0   0 486   0   0   0]
 [  0   0   0 553   0   0   0]
 [  0   0   0 615   0   0   0]
 [  0   0   0 576   0   0   0]]


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 13ms/step


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 12ms/step
rslts= X 4  average= [90 78 89]  max= [90 79 89]  var= [0 0 0]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
rslts

array(['X 1  average= [71 70 70]  max= [72 71 70]  var= [3 0 1]',
       'X 2  average= [92 78 90]  max= [92 79 91]  var= [0 0 1]',
       'X 3  average= [90 79 89]  max= [91 79 90]  var= [0 0 0]',
       'X 4  average= [90 78 89]  max= [90 79 89]  var= [0 0 0]'],
      dtype='<U55')